In [15]:
import os

input_dir = "input"
mp4_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.mp4')])

In [19]:
import os
import cv2
from tqdm import tqdm

input_dir = "input"
intermediate_dir = "intermediate_results2"
output_dir = "output_pip"  # Directory for picture-in-picture videos

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

mp4_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.mp4')])

for file in tqdm(mp4_files, desc="Processing videos", position=0, leave=True):
    try:
        input_path = os.path.join(input_dir, file)
        action_dir = file.replace('.mp4', '')

        if action_dir != "action_200_offence":
            continue
        
        minimap_video_path = os.path.join(intermediate_dir, action_dir, "minimap_augmented.mp4")
        original_video_path = os.path.join(intermediate_dir, action_dir, "player_detections.mp4")

        # Check if original video exists
        if not os.path.exists(original_video_path):
            print(f"Original video not found: {original_video_path}")
            continue
            
        # Create temporary path for video without legend
        no_legend_path = os.path.join(intermediate_dir, action_dir, "player_detections_no_legend.mp4")
        
        # Open original video
        cap = cv2.VideoCapture(original_video_path)
        
        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        # Create video writer for output without legend
        # Original width minus legend width (250), resized to HD (1920x1080)
        output_width = 1920
        output_height = 1080
        out_no_legend = cv2.VideoWriter(no_legend_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (output_width, output_height))
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Crop out the legend part (rightmost 250 pixels)
            frame_no_legend = frame[:, :width-250]
            
            # Resize to HD resolution
            frame_no_legend = cv2.resize(frame_no_legend, (output_width, output_height))
            out_no_legend.write(frame_no_legend)
            
        cap.release()
        out_no_legend.release()
        
        # Update path to use video without legend
        original_video_path = no_legend_path
        
        # Output path for the picture-in-picture video
        output_path = os.path.join(intermediate_dir, action_dir, f"pip.mp4")
        
        # Open both videos
        minimap_cap = cv2.VideoCapture(minimap_video_path)
        original_cap = cv2.VideoCapture(original_video_path)
        
        # Get properties of the videos
        minimap_width = int(minimap_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        minimap_height = int(minimap_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        original_fps = original_cap.get(cv2.CAP_PROP_FPS)
        
        # Set up video writer for output - now using original video dimensions
        fourcc = cv2.VideoWriter_fourcc(*'avc1')
        out = cv2.VideoWriter(output_path, fourcc, original_fps, (original_width, original_height))
        
        # Determine the size of the PIP (now for minimap) - increased from 5 to 4 to make it bigger
        pip_width = original_width // 4  # Changed from 5 to 4 to make minimap bigger
        pip_height = int(minimap_height * pip_width / minimap_width)
        
        # Position for the PIP (bottom right with 10px padding)
        x_offset = original_width - pip_width - 10
        y_offset = original_height - pip_height - 10
        
        while True:
            # Read frames from both videos
            minimap_ret, minimap_frame = minimap_cap.read()
            original_ret, original_frame = original_cap.read()
            
            # If either video ends, break the loop
            if not minimap_ret or not original_ret:
                break
            
            # Resize the minimap frame for PIP
            pip_frame = cv2.resize(minimap_frame, (pip_width, pip_height))
            
            # Create a region of interest on the original frame
            roi = original_frame[y_offset:y_offset+pip_height, x_offset:x_offset+pip_width]
            
            # Create a mask for the PIP and its inverse
            pip_gray = cv2.cvtColor(pip_frame, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(pip_gray, 0, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)
            
            # Black out the area of PIP in ROI
            original_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
            
            # Take only region of PIP from PIP frame
            pip_fg = cv2.bitwise_and(pip_frame, pip_frame, mask=mask)
            
            # Put PIP in ROI and modify the original frame
            dst = cv2.add(original_bg, pip_fg)
            original_frame[y_offset:y_offset+pip_height, x_offset:x_offset+pip_width] = dst
            
            # Write the frame to output
            out.write(original_frame)
        
        # Release resources
        minimap_cap.release()
        original_cap.release()
        out.release()
        
        print(f"Created picture-in-picture video: {output_path}")
        
        break
    
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

Processing videos:  34%|███▍      | 138/400 [00:03<00:06, 40.33it/s]

Created picture-in-picture video: intermediate_results2/action_200_offence/pip.mp4
